# <center >Accidents corporels de la route
### Ouverture de la base de données

Faire les importations nécessaires :

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

Importer les 4 fichiers :

In [2]:
df_carac = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a", sep = ";")
df_lieux = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6", sep = ";")
df_usagers = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028", sep = ";")
df_vehicules = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667", sep = ";")

Une première sélection a été réalisée. Nous avons éliminé les variables qui ne semblaient avoir aucun lien avec la gravité de l'accident.

In [3]:
df_carac = df_carac[["Num_Acc", "jour", "mois", "an", "hrmn", "lum", "dep", "com", "agg", "int", "atm", "col", "lat", "long"]]
df_lieux = df_lieux[["Num_Acc", "catr", "circ", "nbv", "surf", "prof", "plan", "infra", "situ", "vma"]] 
df_usagers = df_usagers[["Num_Acc", "id_vehicule", "place", "catu", "sexe", "grav", "an_nais", "trajet", "secu1"]]
df_vehicules = df_vehicules[["Num_Acc", "id_vehicule", "catv", "obs", "obsm", "choc", "occutc"]]

A l'aide du descriptif des modalités, nous remplaçons les modalités par leur nom complet pour une meilleure compréhension.

In [4]:
# Variables de la base Caracteristiques
def modalite_caracteristiques(df):
    df["lum"]= df['lum'].map({1:'Plein jour',
                        2:'Crépuscule ou aube',
                        3: "Nuit sans éclairage public",
                        4: "Nuit avec éclairage public non allumé",
                        5: "Nuit avec éclairage public allumé"})
    df['agglo'] = df['agg'].map({1:'Hors agglomeration',
                             2:'Agglomeration'})
    del df['agg']
    
    df['int']=df['int'].map({1:'Hors intersection',
                        2:'Intersection en X',
                        3:'Intersection en T',
                        4:'Intersection en Y',
                        5:'Intersection à plus de 4 branches',
                        6:'Giratoire',
                        7:"Place",
                        8:'Passage à niveaux',
                        9:'Autre'})
    df['atm'] = df['atm'].map({1:'Normale',
                         2:'Pluie legere',
                         3:'Pluie forte',
                         4:'Neige',
                         5:'Brouillard',
                         6:'Vent fort',
                         7:'Eblouissant',
                         8:'Couvert',
                         9:'Autre'})
    df['collision'] = df['col'].map({1:'2 - frontale',
                                 2:'2-arriere',
                                 3:'2-cote',
                                 4:'3-chaine',
                                 5:'3-multiple',
                                 6:'autre',
                                 7:'Sans collision'})
    del df['col']

# Variables de la base Usagers
def modalite_usagers(df) :
    df['catu'] = df['catu'].map({1:'Conducteur',
                                 2:'Passager',
                                 3:'Pieton'})
    df['sexe'] = df['sexe'].map({1:'Homme',
                                 2:'Femme'})
    df['grav'] = df['grav'].map({1:'Indemne',
                                 2:'Tue',
                                 3:'Blesse hospitalise',
                                 4:'Blesse leger'})

    # Variables de la base Lieux
def modalite_lieux(df) :
    df['catr'] = df['catr'].map({1:'Autoroute',
                                 2:'Route nationale',
                                 3:'Route départementale',
                                 4:'Voie Communale',
                                 5:'Hors réseau public',
                                 6:'Parc de stationnement',
                                 7:'Routes de métropole urbaine',
                                 9:'Autre'})
    df["circ"]=df["circ"].map({1:"A sens unique",
                              2:"Bidirectionnelle",
                              3:'A chaussées séparées',
                              4:"Avec voies d'affectation variable"})
    df["prof"]=df["prof"].map({1:"Plat",
                              2:"Pente",
                              3:'Sommet de côte',
                              4:"Bas de côte"})
    df["plan"]=df["plan"].map({1:"Rectiligne",
                              2:"En courbe à gauche",
                              3:'En courbe à droite',
                              4:"En S"})
    df['surf'] = df['surf'].map({1:'Normale',
                                 2:'Mouillée',
                                 3:'Flaques',
                                 4:'Inondée',
                                 5:'Enneigée',
                                 6:'Boue',
                                 7:'Verglecée',
                                 8:'Corps gras - huile',
                                 9:'Autre'})
    df['infra'] = df['infra'].map({0:'Aucun',
                                1:'Souterrain-tunnel',
                                 2:'Pont',
                                 3:'Bretelle',
                                 4:'Voie ferrée',
                                 5:'Carrefour aménagé',
                                 6:'Zone piétonne',
                                 7:'Zone de péage'})
    df['situ'] = df['situ'].map({1:'Sur chaussée',
                                 2:"Sur bande d'arrêt d'urgence",
                                 3:'Sur accotement',
                                 4:'Sur trottoir',
                                 5:'Sur piste cyclable',
                                 6:'Sur autre voie spéciale',
                                 8:'Autre'})

# Variables de la base Véhicules
def modalite_vehicules(df) :
    df['catv'] = df['catv'].map({0:'Autre',
                             1:'EDP',
                             2:'Deux roues',
                             3:'Voiture',
                             4:'Deux roues',
                             5:'Deux roues',
                             6:'Autre',
                             7:'Voiture',
                             8:'Voiture',
                             9:'Voiture',
                             10:'Voiture',
                             11:'Voiture',
                             12:'Voiture',
                             13:'Poids lourd',
                             14:'Poids lourd',
                             15:'Poids lourd',
                             16:'Poids lourd',
                             17:'Poids lourd',
                             18:'Transport en commun',
                             19:'Transport en commun',
                             20:'Autre',
                             21:'Autre',
                             30:'Deux roues',
                             31:'Deux roues',
                             32:'Deux roues',
                             33:'Deux roues',
                             34:'Deux roues',
                             35:'Autre',
                             36:'Autre',
                             37:'Transport en commun',
                             38:'Transport en commun',
                             39:'Transport en commun',
                             40:'Transport en commun',
                             41:'Deux roues',
                             42:'Deux roues',
                             43:'Deux roues',
                             50:'EDP',
                             60:'EDP',
                             80:'EDP',
                             99:'Autre'})
    df["choc"] = df["choc"].map({0: 'Aucun',
                                1:'Avant',
                                 2:"Avant droit",
                                 3:'Avant gauche',
                                 4:'Arrière',
                                 5:'Arrière droit',
                                 6:'Arrière gauche',
                                 7:'Côté droit',
                                 8:'Côté gauche',
                                 9:'Chocs multiples'})
                             
modalite_caracteristiques(df_carac)
modalite_usagers(df_usagers)
modalite_lieux(df_lieux)
modalite_vehicules(df_vehicules)

Fusionner les 4 bases de données :

In [5]:
df = df_carac.merge(df_lieux, how='left', on=['Num_Acc'])
df = df.merge(df_vehicules, how='inner', on=['Num_Acc'])
df = df.merge(df_usagers, how='inner', on=['Num_Acc', 'id_vehicule'])

Au total, en 2019, il y eu 58 840 accidents corporels de route qui ont concernés au total 100 710 véhicules et 132 977 personnes. \
Ainsi, dans notre table globale, nous avons 132 977 observations et 36 variables.

### Appropriation du jeu de données et structure de la base

In [50]:
#df.columns

In [51]:
#df.head(5)

In [27]:
#df.dtypes

In [28]:
#print(df.describe(include='all'))

### Etude des valeurs manquantes :

Dans ce jeu de données, les valeurs non renseignées sont indiquées par "-1". \
La majorité des variables ne possèdent pas de valeurs manquantes. \
Quelques variables possèdent moins d'1% de valeurs manquantes (ex : collision, état de la surface). \
Concernant la variable "Circulation", il y a 5.06% sont des valeurs manquantes. 

In [64]:
var_carac = ["lum", "agglo", "int", "collision", "atm", 'jour', 'mois', 'hrmn', 'dep', 'com', 'lat', 'long']
var_lieux = ["catr", "circ", "surf", "prof", "plan", "infra", "situ", 'nbv', 'vma']
var_usagers = ["place", "catu", "sexe", "grav", "an_nais"]
var_vehicules = ["catv", 'obs', 'obsm', 'choc', 'occutc']

def effecitf_val_manquantes(df, colonne):
    for elem in colonne :
        eff = (df[elem].isna().sum()).round(2)
        freq = ((df[elem].isna().sum())/len(df[elem])*100).round(2)
        print("Pour la variable " + elem + " :  " + freq.astype(str) + " % \n ce qui représente un effectif de :    " + eff.astype(str))
        print("\n")
#effecitf_val_manquantes(df_carac, var_carac)
#effecitf_val_manquantes(df_lieux, var_lieux)
#effecitf_val_manquantes(df_usagers, var_usagers)
#effecitf_val_manquantes(df_vehicules, var_vehicules)

### Etude des modalités rares :

In [7]:
var_carac = ["lum", "agglo", "int", "collision", "atm"]
var_lieux = ["catr", "circ", "surf", "prof", "plan", "infra", "situ"]
var_usagers = ["place", "catu", "sexe", "grav"]
var_vehicules = ["catv", 'choc']
def effectif_pourcentage(df, colonne):
    for elem in colonne :
        print("Pour la variable " + elem + " :")
        print((df[elem].value_counts())/len(df[elem])*100)
        print("\n")
#effectif_pourcentage(df_carac, var_carac)
#effectif_pourcentage(df_lieux, var_lieux)
#effectif_pourcentage(df_usagers, var_usagers)
#effectif_pourcentage(df_vehicules, var_vehicules)

En statistique, il n'est pas souhaitable d'avoir des modalités rares qui ont un effectif inférieur à 5%. Pour cela, nous avons regroupé les modalités entre elles en suivant deux méthodes différentes. Dans un premier temps, nous avons regroupé des modalités entre elles lorsqu'elles avaient un sens proche. Par exemple, "Intersection en X" et "Intersection en Y" correspondent à la modalité commune "Intersection".

In [66]:
# La modalité "Avec éclairage non allumé" devient "Sans éclairage"
df['lum']=df['lum'].replace(["Nuit avec éclairage public non allumé"], "Nuit sans éclairage public")

# Variable intersection : on regroupe tous les types d'intersections
df['int']=df['int'].replace(["Intersection en X", "Intersection en Y", "Intersection en T"], "Intersection")

# Variable catr :


# Variable circulation : 
df['circ']=df['circ'].replace(["Avec voies d'affectation variable"], "Bidirectionnelle")

# Prof :
df['prof'] = df['prof'].replace({"Bas de côte" : "Sommet de côte"})

# Plan : 
df['plan'] = df['plan'].replace(["En courbe à gauche", "En courbe à droite", "En S"], "Courbe")

La seconde méthode est de regrouper les modalités représentant moins de 5% entre elles afin de créer une modalité "Autre".

In [67]:
var_carac = ["lum", "agglo", "int", "collision", "atm"]
var_lieux = ["catr", "circ", "surf", "prof", "plan", "infra", "situ"]
var_usagers = ["catu", "sexe", "grav"]
var_vehicules = ["catv", 'choc']

def regroupement_modalites(df, colonne):
    for elem in colonne :
        frequence = df[elem].value_counts()/len(df[elem])*100
        for i in range(len(frequence)) :
            if frequence[i] < 5 :
                df.loc[df[elem]==frequence.index[i], elem] = "Autre"   
                
regroupement_modalites(df_carac, var_carac)
regroupement_modalites(df_lieux, var_lieux)
regroupement_modalites(df_usagers, var_usagers)
regroupement_modalites(df_vehicules, var_vehicules)
#effectif_pourcentage(df_carac, var_carac)
#effectif_pourcentage(df_lieux, var_lieux)
#effectif_pourcentage(df_usagers, var_usagers)
#effectif_pourcentage(df_vehicules, var_vehicules)

## Ajout de variables 

In [68]:
df["age_continu"]= 2019 - df["an_nais"]

On crée une variable 'obstacle' liée à l'accident

In [69]:
conditions_obstacle = [
                        (df['obs']>0) & (df['obsm']>0),
                        (df['obs']>0) & (df['obsm']<=0),
                        (df['obs']<=0) & (df['obsm']>0),
                        (df['obs']==0) & (df['obsm']==0)]

types_obstacle = ['1 obstacle mobile et 1 obstacle fixe', '1 obstacle fixe', '1 obstacle mobile', 'Pas dobstacle']

df['obstacle'] = np.select(conditions_obstacle, types_obstacle, default='NaN')

On crée ensuite une nouvelle variable de catégorie d'âge, liée aux usagers, qu'on appelle 'age', à partir de la variable existante 'an_nais' (année de naissance)

In [70]:
conditions_age = [
                    (0 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 15),
                    (16 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 30),
                    (31 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 65),
                    (66 <= 2019 - df['an_nais'])]

categories_age = ['enfant', 'jeune', 'adulte', 'personne agee']

df['age'] = np.select(conditions_age, categories_age, default='NaN')

#### Variable gravite

On ajoute une variable qui donne une note de gravité à chaque accident nommée "gravite". 
- gravite = 0 si tout le monde ressort indemne de l'accident 
- gravite = 1 si il y a au moins un blessé léger 
- gravite = 2 si il y a au moins un blessé hospitalisé ou au moins un mort 

In [8]:
def calcul_gravite(l_grav):
    # prend en entrée la liste des grav d'un même accident l_grav
    note = 0
    if (('Tue' in l_grav) or ('Blesse hospitalise' in l_grav)):
        #on check si on a un 'Tue' ou un 'Blesse hospitalise' (var grav) => note=2
        note = 2
        return (note) 
    elif ('Blesse leger' in l_grav):
        #si non on check si on a un 'Blesse leger' => note = 1
        note = 1
        return (note)
    # si non => note = 0
    return(note)

#TEST OK 
#calcul_gravite(['Indemne','Blesse hospitalise','Blesse leger','Tue'])
#calcul_gravite(['Indemne','Blesse leger','Indemne'])
#calcul_gravite(['Indemne','Indemne','Indemne'])
#calcul_gravite([])

In [9]:
#on veut récupérer toutes les données de gravité pour chaque personne d'un meme accident (var Num_Acc)
#on crée une variable gravite remplie de zero dans df 
df = df.assign(gravite=0)

#on récupère tous les Num_Acc distincts
accidents = df['Num_Acc'].unique()

for num in accidents: 
    #on boucle sur les Num_Acc pour récupérer toutes les valeurs de grav pour un meme accident 
    l_grav = list(df[df['Num_Acc'] == num]['grav'])
    note = calcul_gravite(l_grav) #on applique pour chaque accident la fonction calcul_gravite 
    for i in df[df['Num_Acc'] == num].index:
        #on insert la valeur dans chaque case de la nouvelle variable 
        df['gravite'][i] = note

/Users/cloclo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1]:
#print(df)